In [1]:
%%bash
dx download --recursive -f /mhc_alt/
dx download /FastaReader.py

In [2]:
from FastaReader import FastaReader
import glob
from collections import Counter

In [3]:
seqs = {}
fns = glob.glob("mhc_alt/G*")
for f in fns:                
    f = FastaReader(f)
    for r in f:
        print(r.name.split()[0])
        seqs[r.name.split()[0]] = r.sequence

GL000251.2
GL000250.2
GL000256.2
GL000255.2
GL000254.2
GL000253.2
GL000252.2


In [4]:
kmer_set = {}
k=128
for n, seq in seqs.items():
    kmers = []
    for i in range(len(seq)-k):
        kmer = seq[i:i+k]
        kmers.append(kmer)
    count = Counter(kmers)
    
    kmer_set[n] = set([hash(x[0]) for x in count.items() if x[1] == 1])
 


In [5]:
ss = []
for n, s in kmer_set.items():
    ss.extend(s)
count = Counter(ss)
ss = set([x[0] for x in count.items() if x[1] > 2])


In [6]:
len(ss)

4447506

In [7]:
%%capture
%%bash
conda install -y numpy matplotlib networkx

In [8]:
%matplotlib inline

In [9]:
import matplotlib.pyplot as plt

In [10]:
import networkx as nx

In [11]:
import random


In [12]:
ss=set(random.sample(list(ss),1000))

In [13]:
len(ss)

1000

In [14]:
G=nx.DiGraph()

In [15]:
kp2seg = {}
for n, seq in seqs.items():
    klist = []
    for i in range(len(seq)-k):
        kmer = seq[i:i+k]
        if hash(kmer) in ss:
            klist.append( (hash(kmer),i) )
    for i in range(len(klist)-1):
        key = klist[i][0], klist[i+1][0]
        seg = klist[i][1], klist[i+1][1]
        kp2seg.setdefault(key, [])
        kp2seg[key].append((n, seg))
        G.add_edge(klist[i][0], klist[i+1][0])
        

In [16]:
nx.write_gexf(G, "test.gexf")

In [17]:
len(G.nodes), len(G.edges)

(1000, 1405)

In [18]:
def get_graph(k, thresholds, fn):
    kmer_set = {}
    for n, seq in seqs.items():
        kmers = []
        for i in range(len(seq)-k):
            kmer = seq[i:i+k]
            kmers.append(kmer)
        count = Counter(kmers)

        kmer_set[n] = set([hash(x[0]) for x in count.items() if x[1] == 1])
 
    ss = []
    for n, s in kmer_set.items():
        ss.extend(s)
    count = Counter(ss)
    ss = set([x[0] for x in count.items() if x[1] > thresholds])
    ss = set(random.sample(list(ss),1000))
    G=nx.DiGraph()
    
    kp2seg = {}
    for n, seq in seqs.items():
        klist = []
        for i in range(len(seq)-k):
            kmer = seq[i:i+k]
            if hash(kmer) in ss:
                klist.append( (hash(kmer),i) )
        for i in range(len(klist)-1):
            key = klist[i][0], klist[i+1][0]
            seg = klist[i][1], klist[i+1][1]
            kp2seg.setdefault(key, [])
            kp2seg[key].append((n, seg))
            G.add_edge(klist[i][0], klist[i+1][0])
    nx.write_gexf(G, fn)


In [ ]:
for k in range(24, 128, 32):
    for t in range(2,8,2):
        get_graph(k, t, f"g_{k}_{t}.gexf")